In [1]:
using MLJ

In [2]:
using Statistics, DataFrames
x1 = rand(300)
x2 = rand(300)
x3 = rand(300)
y = exp.(x1 - x2 -2x3 + 0.1*rand(300))
X = DataFrame(x1=x1, x2=x2, x3=x3) # a column table
ys = source(y)
Xs = source(X)


Source @ 9…69

In [3]:
stand_model = Standardizer()
stand = machine(stand_model, Xs)

NodalMachine @ 3…79 = machine(Standardizer @ 3…14, 9…69)

In [4]:
W = transform(stand, Xs)

Node @ 1…71 = transform(3…79, 9…69)

In [5]:
test, train = partition(eachindex(y), 0.8)
fit!(W, rows=train)
W()           # transform all data
W(rows=test ) # transform only test data
W(X[3:4,:])   # transform any data, new or old


┌ Info: Training NodalMachine @ 3…79.
└ @ MLJ /Users/davidbuchacaprats/.julia/packages/MLJ/gsBfz/src/machines.jl:115


,x1,x2,x3
,Float64,Float64,Float64
1,1.31766,-1.08237,0.455134
2,0.0386412,0.390039,1.64516


In [6]:
box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
box = machine(box_model, ys)
z = transform(box, ys)

ridge_model = RidgeRegressor(lambda=0.1)
ridge =machine(ridge_model, W, z)
zhat = predict(ridge, W)

yhat = inverse_transform(box, zhat)

Node @ 7…36 = inverse_transform(9…40, predict(1…70, transform(3…79, 9…69)))

In [7]:
fit!(yhat, rows=train)


┌ Info: Not retraining NodalMachine @ 3…79. It is up-to-date.
└ @ MLJ /Users/davidbuchacaprats/.julia/packages/MLJ/gsBfz/src/networks.jl:227
┌ Info: Training NodalMachine @ 9…40.
└ @ MLJ /Users/davidbuchacaprats/.julia/packages/MLJ/gsBfz/src/machines.jl:115
┌ Info: Training NodalMachine @ 1…70.
└ @ MLJ /Users/davidbuchacaprats/.julia/packages/MLJ/gsBfz/src/machines.jl:115


Node @ 7…36 = inverse_transform(9…40, predict(1…70, transform(3…79, 9…69)))

In [8]:
rms(y[test], yhat(rows=test)) # evaluate


0.1033033675447962

### Exporting a laerning network as a stand-alone model

In [17]:
mutable struct WrappedRidge <: DeterministicNetwork
    ridge_model
end


ErrorException: invalid redefinition of constant WrappedRidge

In [18]:

# keyword constructor
WrappedRidge(; ridge_model=RidgeRegressor) = WrappedRidge(ridge_model); 

In [19]:
WrappedRidge

WrappedRidge (generic function with 1 method)